In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
import datetime
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import pandas as pd
pd.options.display.max_columns=60

Augmentacja tylko trenigu

EfficentNet - zostajemy

https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
keras.iokeras.io

https://albumentations.ai/

GAN dostrajanie stylu

IoU - dla nas się nie sprawdzi tylko zdjęcia ptaków

Warstwy ilosc nueronów rosnie/maleje

Ile i jakich agumtancji do jakiej ilośc zdjęci dążymy i czy muszą być równomiernie rozłożone?

Ilość neroów ukrytej do ilosc wejsc/wyjsc?

525 klas i na poziomie apliacji przekazana informracja o statusie.

Metryka porawność rozpoznawani prawidłowego do wszystkich klas i fałszywie pozytwyne będą mieć większą wartość w celu ochrony ptaka.

Gdzie można wykorzystać aby zapilkować gotowy model.


## Importing two data sets:

+ df_status is data from ICUN with information about status protection of bird.
+ df_lp is data with information about images path,which set it is (train,valid,test)

##### Join will be done base on scientiffic name because it is unique for species and can be used to join both data frames.

In [2]:
df_status = pd.read_csv(r"C:\Users\Data\Documents\git\JDSZR11-chubs_chubs\Birds Image Clasification\IUCN\assessments.csv")

In [3]:
df_lp = pd.read_csv(r"C:\Users\Data\Documents\git\JDSZR11-chubs_chubs\Birds Image Clasification\archive (1)\birds.csv")

In [4]:
df_lp.head()

,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI


In [5]:
df_status.head()

,assessmentId,internalTaxonId,scientificName,redlistCategory,redlistCriteria,yearPublished,assessmentDate,criteriaVersion,language,rationale,habitat,threats,population,populationTrend,range,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes
0,84694321,22679393,Polyplectron schleiermacheri,Endangered,C2a(i),2016,2016-10-01 00:00:00 UTC,3.1,English,This elusive species's status is difficult to ...,Its ecological needs are poorly understood. An...,"In central Kalimantan, habitat loss, degradati...","The population is estimated to number 1,000-2,...",Decreasing,<em>Polyplectron schleiermacheri</em> is endem...,NaN,Terrestrial,<strong>Conservation Actions Underway</strong>...,Indomalayan,NaN,False,False,Global
1,89641304,22720960,Emberiza rustica,Vulnerable,A2abcd+3bcd+4abcd,2016,2016-10-01 00:00:00 UTC,3.1,English,This species has been uplisted to Vulnerable a...,The species breeds in swampy lowland spruce (<...,The numbers of this species have declined in t...,"The European population is estimated at 681,00...",Decreasing,This bunting breeds across northern latitudes ...,NaN,Terrestrial|Freshwater (=Inland waters),<strong>Conservation and research actions unde...,Indomalayan|Nearctic|Palearctic,NaN,False,False,Global
2,90341807,22728245,Chlamydotis undulata,Vulnerable,A2cd+3cd+4cd,2016,2016-10-01 00:00:00 UTC,3.1,English,This recently split species is listed as Vulne...,It inhabits sandy and stony semi-desert and is...,"The principal threat is from hunting, which ha...","In the mid-1990s, this species's population wa...",Decreasing,This species&#160;occurs across a wide range i...,NaN,Terrestrial,<strong>Conservation Actions Underway</strong>...,Afrotropical|Palearctic,NaN,False,False,Global
3,92761019,22678194,Crypturellus kerriae,Vulnerable,"B1ab(i,ii,iii,v);C2a(i)",2016,2016-10-01 00:00:00 UTC,3.1,English,This species is Vulnerable because it is known...,It occurs in humid primary forest. The known s...,Vast areas of seemingly suitable habitat remai...,"The population is estimated to number 2,500-9,...",Decreasing,<em>Crypturellus kerriae</em> is a poorly-know...,NaN,Terrestrial,<strong>Conservation Actions Underway</strong>...,Neotropical,NaN,False,False,Global
4,92761578,22678217,Crypturellus noctivagus,Near Threatened,A2cd+3cd+4cd,2016,2016-10-01 00:00:00 UTC,3.1,English,This species is classified as Near Threatened ...,"It occurs in caatinga and Atlantic forest, par...",It suffers from widespread and continuing habi...,The global population size has not been quanti...,Decreasing,<em>Crypterellus noctivagus</em> is rare to lo...,NaN,Terrestrial,<strong>Conservation Actions Underway</strong>...,Neotropical,NaN,False,False,Global


In [6]:
## adjusting name of columns to scientificName for unifying
df_lp = df_lp.rename(columns={'scientific name' : 'scientificName'})
df_lp['scientificName'] = df_lp['scientificName'].apply(lambda  x : x.lower())
df_status['scientificName'] = df_status['scientificName'].apply(lambda  x : x.lower())

In [7]:
df_status = df_status.set_index(['scientificName'])

In [8]:
df = df_lp.join(df_status,on='scientificName')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89885 entries, 0 to 89884
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   class id                  89885 non-null  float64
 1   filepaths                 89885 non-null  object 
 2   labels                    89885 non-null  object 
 3   data set                  89885 non-null  object 
 4   scientificName            89885 non-null  object 
 5   assessmentId              14650 non-null  float64
 6   internalTaxonId           14650 non-null  float64
 7   redlistCategory           14650 non-null  object 
 8   redlistCriteria           14650 non-null  object 
 9   yearPublished             14650 non-null  float64
 10  assessmentDate            14650 non-null  object 
 11  criteriaVersion           14650 non-null  float64
 12  language                  14650 non-null  object 
 13  rationale                 14650 non-null  object 
 14  habita

In [10]:
# droping unwated columns
list_col = list(df.iloc[:,8:].columns) + ['assessmentId','internalTaxonId']
df = df.drop(columns=list_col)

In [43]:
# extract from directory datasets train,test,valid
train = tf.keras.utils.image_dataset_from_directory(
    directory=r'archive (1)\train',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)
test = tf.keras.utils.image_dataset_from_directory(
    directory=r'archive (1)\test',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)
valid = tf.keras.utils.image_dataset_from_directory(
    directory=r'archive (1)\valid',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 84635 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.


In [12]:
## extracting classes from train set
class_names_train = train.class_names
print(class_names_train)

['ABBOTTS BABBLER', 'ABBOTTS BOOBY', 'ABYSSINIAN GROUND HORNBILL', 'AFRICAN CROWNED CRANE', 'AFRICAN EMERALD CUCKOO', 'AFRICAN FIREFINCH', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN PYGMY GOOSE', 'ALBATROSS', 'ALBERTS TOWHEE', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'ALTAMIRA YELLOWTHROAT', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN DIPPER', 'AMERICAN FLAMINGO', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART', 'AMERICAN ROBIN', 'AMERICAN WIGEON', 'AMETHYST WOODSTAR', 'ANDEAN GOOSE', 'ANDEAN LAPWING', 'ANDEAN SISKIN', 'ANHINGA', 'ANIANIAU', 'ANNAS HUMMINGBIRD', 'ANTBIRD', 'ANTILLEAN EUPHONIA', 'APAPANE', 'APOSTLEBIRD', 'ARARIPE MANAKIN', 'ASHY STORM PETREL', 'ASHY THRUSHBIRD', 'ASIAN CRESTED IBIS', 'ASIAN DOLLARD BIRD', 'ASIAN GREEN BEE EATER', 'ASIAN OPENBILL STORK', 'AUCKLAND SHAQ', 'AUSTRAL CANASTERO', 'AUSTRALASIAN FIGBIRD', 'AVADAVAT', 'AZARAS SPINETAIL', 'AZURE BREASTED PITTA', 'AZURE JAY', 'AZURE TANAGER', '

In [13]:
#creating lables for NNm
class_names_train = df['labels'].unique()

In [14]:
class_names_train = OneHotEncoder().fit_transform(class_names_train.reshape(-1,1)).toarray()

In [15]:
# Checkinkg batch information labels,img height, img width,colors and nubmer of labels
for image_batch, labels_batch in train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


In [16]:
# creating normalization layer to noramlize numbers in image
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [17]:
normalized_ds = train.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[3]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [18]:
AUTOTUNE = tf.data.AUTOTUNE

train_buff = train.cache().prefetch(buffer_size=AUTOTUNE)
valid_buff = valid.cache().prefetch(buffer_size=AUTOTUNE)

In [19]:
num_class = 525

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_class)
])

In [20]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [21]:
model.fit(
  train,
  validation_data=valid,
  batch_size=32,
  epochs=1
)

   2/2645 [..............................] - ETA: 2:18:30 - loss: 40.5190 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [30]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

In [31]:
base_model.trainable = False

In [32]:
input_shape = (224,224,3)

In [33]:
inputLayer = tf.keras.Input(shape=(input_shape))

In [34]:
x = base_model(inputLayer, training=False)
x = layers.GlobalAveragePooling2D()(x)
output = layers.Dense(525)(x)

In [35]:
model = tf.keras.Model(inputLayer, output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 525)               672525    
                                                                 
Total params: 4,722,096
Trainable params: 672,525
Non-trainable params: 4,049,571
_________________________________________________________________


In [36]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

In [40]:
model.fit(
  train,
  validation_data=valid,
  epochs=1,
  batch_size = 256
)

2645/2645 [==============================] - 3387s 1s/step - loss: 6.2447 - accuracy: 0.0110 - val_loss: 6.2409 - val_accuracy: 0.0122
